## Three Factor Model (Fama and French)

O objetivo desse notebook é replicar o modelo de três fatores de Fama e French. Para isso seguiremos o tutorial encontrado na internet [Estimating Stock Returns with Fama-French Three-Factor Model in Python](https://medium.com/the-handbook-of-coding-in-finance/estimating-stock-returns-with-fama-french-three-factor-model-in-python-1a98e3936859).

O modelo de três fatores de Fama e French é amplamente usado na academia e na industria em busca de estimar melhor o excesso de retorno de ativos financeiros. O modelo nada mais é do que uma extensão do [Capital Asset Pricing Model (CAPM)](https://en.wikipedia.org/wiki/Capital_asset_pricing_model) que é um modelo de precificação de ativos através do excesso de retorno esperado de mercado, isso é,

$$
\mathbb{E}[R_i] = R_f + \beta_i \left( \mathbb{E}[R_m] - R_f \right)
$$

onde 

* $\mathbb{E}[R_i]$ é o retorno esperado do ativo $i$
* $R_f$ é o retorno do ativo livre de risco
* $\mathbb{E}[R_m]$ é o retorno esperado do mercado
* $\beta_i = \frac{\mathbb{C}[R_i, R_m]}{\mathbb{V}[R_m]}$

O modelo de Fama e French se extende adicionando mais dois fatores ao CAPM original. Enquanto o CAPM utiliza apenas o Market Factor (MKT), $\mathbb{E}[R_m] - R_f$, também conhecido como excesso de retorno de mercado, o modelo de Fama e French adiciona um componentee de tamanho, Size Factor (SMB) e um componente de valor, Value Factor (HML), construídos, respectivamente, pelo excesso de retorno entre ativos com pequeno Market Cap e ativos com grande Market Cap e o excesso de retorno entre ativos com alta razão book-to-market e ativos com tal razão baixa.

$$
\mathbb{E}[R_i] = R_f + \beta_1 \operatorname{MKT} + \beta_2 \operatorname{SMB} + \beta_3 \operatorname{HML} + \alpha
$$

O exercício proposto é implementar o Fama-French Three-Factor Model estimando os retornos mensais das ações da Microsoft.

In [1]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff

### Dados

In [2]:
# dados dos preços da ação da Microsoft
ticker = 'msft'
start = '2016-8-31'
end = '2021-8-31'

stock_data = yf.download(ticker, start, end, adjusted=True)

[*********************100%***********************]  1 of 1 completed


In [3]:
stock_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-08-31,57.650002,57.799999,57.299999,57.459999,52.573933,20860300
2016-09-01,57.009998,57.820000,57.009998,57.590000,52.692886,26075400
2016-09-02,57.669998,58.189999,57.419998,57.669998,52.766068,18900500
2016-09-06,57.779999,57.799999,57.209999,57.610001,52.711178,16278400
2016-09-07,57.470001,57.840000,57.410000,57.660000,52.756924,17493400
...,...,...,...,...,...,...
2021-08-24,305.019989,305.649994,302.000000,302.619995,299.263184,18175800
2021-08-25,304.299988,304.589996,300.420013,302.010010,298.660004,20006100
2021-08-26,300.989990,302.429993,298.950012,299.089996,295.772369,17666100


Como construir os três fatores de [Fama e French](https://github.com/guimasuko/three_factor_model/blob/main/Common%20risk%20factors%20in%20the%20returns%20on%20stocks%20and%20bonds.pdf):

* **Market Factor (MKT)**
O Market Factor é o excesso de retorno esperado do mercado em relação ao retorno do ativo livre de risco e pode ser criado a partir da fórmula a seguir

$$
\operatorname{MKT} = \mathbb{E}[R_m] - R_f
$$

Para construir os fatores SMB e HML precisamos categorizar os ativos de duas maneiras diferentes. A primeira clusteriza dois grupos baseado no Market Cap, isso é, categorizamos os ativos em Big (B) ou Small (S). Ativos Big são aqueles no topo de $90\%$ de capitalização, enquanto ativos Small são aqueles com os $10\%$ restante de capitalização. A segunda maneira divide os ativos em três grupos baseado no Book-to-Market Equity (B/M). Os ativos High (H), Middle (M) e Low (L) são respectivamente, aqueles com os $30\%$ maiores B/M, $40\%$ no meio e $30\%$ inferiores. 

Com os ativos categorizados dessa forma, somos capazes de criar seis portifólios diferentes: $\operatorname{S/L}, \operatorname{S/M}, \operatorname{S/H}, \operatorname{B/L}, \operatorname{B/M}, \operatorname{B/H}$ onde, por exemplo, $\operatorname{S/L}$ é o portfólio que contém as ações que estão tanto no grupo de Small Market-Cap quanto no grupo de Low B/M, e o portfólio $\operatorname{B/H}$ contém as ações que estão no grupo de Big Market-Cap e High B/M.

* **Size Factor (SMB)**

O Size Factor é o excesso de retorno ponderado dos portfólios das ações de empresas cujo Market-Cap estão abaixo de 10\% (Small-Cap) em relação ao retorno ponderado dos portfólios das ações de empresas cujo Market-Cap estão acima de 90\% (Large-Cap). O SMB é construído a partir

$$
\operatorname{SMB} = \frac{1}{3}\left( \operatorname{S/L} + \operatorname{S/M} + \operatorname{S/H} \right) - \
                     \frac{1}{3}\left( \operatorname{B/L} + \operatorname{B/M} + \operatorname{B/H} \right)
$$

* **Value Factor (HML)**

O Value Factor é o excesso de retorno ponderado dos portfólio das ações de empresas cujo B/M estão acima do percentil 30 em relação ao retorno ponderado dos portfólios das ações de empresas cujo B/M estão abaixo do percentil 70.

$$
\operatorname{HML} = \frac{1}{2}\left( \operatorname{S/H} + \operatorname{B/H} \right) - \
                     \frac{1}{2}\left( \operatorname{S/L} + \operatorname{B/L} \right)
$$

Para obter os três fatores de Fama-French usaremos uma biblioteca pronta cujo código abaixo retorna um dataframe contendo os três fatores e o retorno do ativo livre de risco. Detalhes adicionais sobre a biblioteca pode ser obtido em [vashOnGitHub](https://github.com/vashOnGitHub/getFamaFrenchFactors).

In [4]:
# importando as colunas dos três fatores e do ativo livre de risco 
ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)

In [5]:
ff3_monthly

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-31,0.0296,-0.0256,-0.0243,0.0022
1926-08-31,0.0264,-0.0117,0.0382,0.0025
1926-09-30,0.0036,-0.0140,0.0013,0.0023
1926-10-31,-0.0324,-0.0009,0.0070,0.0032
1926-11-30,0.0253,-0.0010,-0.0051,0.0031
...,...,...,...,...
2022-08-31,-0.0377,0.0139,0.0031,0.0019
2022-09-30,-0.0935,-0.0082,0.0003,0.0019
2022-10-31,0.0783,0.0010,0.0805,0.0023


Para os preços dos ativos da Microsoft usaremos o preço ajustado de fechamento, pois esse é capaz de capturar pagamentos de dividendos, etc. Também precisamos colocar tal base em periodicidade mensal. Por fim, também precisamos transformar os valores de preços para taxas de retorno.

In [6]:
# resample(): determina a periodicidade
# last(): determina qual preço é pego dentro dessa nova periodicidade
# pct_change(): toma os valores de variação percentual
stock_returns = stock_data['Adj Close'].resample('M').last().pct_change().dropna()

In [7]:
# renomeando o nome da coluna
stock_returns.name = "R_MSFT"

In [8]:
stock_returns.head()

Date
2016-09-30    0.002437
2016-10-31    0.040278
2016-11-30    0.012468
2016-12-31    0.031198
2017-01-31    0.040393
Freq: M, Name: R_MSFT, dtype: float64

In [9]:
# unindo a coluna dos retornos mensais da Microsoft ao dataframe com os fatores de Fama-French (ao unir pela coluna Date, dropamos todas linhas de fatores FF onde não temos dados de retorno da Microsoft)
ff_data = ff3_monthly.merge(stock_returns,on='Date')

In [10]:
ff_data.head()

,Mkt-RF,SMB,HML,RF,R_MSFT
Date,,,,,
2016-09-30,0.0025,0.0213,-0.0123,0.0002,0.002437
2016-10-31,-0.0202,-0.0442,0.0412,0.0002,0.040278
2016-11-30,0.0486,0.0567,0.0819,0.0001,0.012468
2016-12-31,0.0181,0.0008,0.0356,0.0003,0.031198
2017-01-31,0.0194,-0.0114,-0.0276,0.0004,0.040393


### Estimação

Vamos estimar os coeficientes através de OLS 

$$
y = \theta X
$$

onde 

* $y = \mathbb{E}[R_i] - R_f$
* $X = \left(1, \operatorname{MKT}, \operatorname{SMB}, \operatorname{HML} \right)'$
* $\theta = \left(\alpha, \beta_1, \beta_2, \beta_3 \right)'$



In [11]:
# criando as variáveis
X = ff_data[['Mkt-RF', 'SMB', 'HML']]
X = sm.add_constant(X)
y = ff_data['R_MSFT'] - ff_data['RF']

In [12]:
# resultados do modelo
ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     28.99
Date:                Wed, 01 Feb 2023   Prob (F-statistic):           1.91e-11
Time:                        16:30:23   Log-Likelihood:                 126.51
No. Observations:                  60   AIC:                            -245.0
Df Residuals:                      56   BIC:                            -236.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0155      0.004      3.670      0.0

Encontramos que todos coeficientes são estatísticamente significantes com p-value menor que 0.05. Outro resultado é que o coeficiente do excesso de retorno do mercado é menor que um, isso significa que o ativo é menos volátil que o mercado.

In [13]:
# parâmetros
a, b1, b2, b3 = ff_model.params

Para finalizar o exercício, o autor toma a média mensal de cada um dos três fatores, e também do retorno do ativo livre de risco e aplica os parâmetros encontrados para encontrar o retorno médio mensal da ação da Microsoft.

In [14]:
# criando as médias dos fatores
rf = ff_data['RF'].mean()
market_premium = ff3_monthly['Mkt-RF'].mean()
size_premium = ff3_monthly['SMB'].mean()
value_premium = ff3_monthly['HML'].mean()

In [15]:
# estimando o retorno esperado mensal
expected_monthly_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium 

In [19]:
print("Retorno esperado mensal da ação da Microsoft: " + str(round(expected_monthly_return, 4)))

Retorno esperado mensal da ação da Microsoft: 0.0049


In [17]:
expected_yearly_return = expected_monthly_return * 12

In [20]:
print("Retorno esperado anual da ação da Microsoft: " + str(round(expected_yearly_return, 4)))

Retorno esperado anual da ação da Microsoft: 0.0583
